In [1]:
# 
import sys, os
import numpy as np

sys.path.append("../yolov5/") 

In [2]:
import torch

from utils.loss import ComputeLoss
from models.common import DetectMultiBackend
from utils.general import (
    LOGGER,
    TQDM_BAR_FORMAT,
    Profile,
    check_dataset,
    check_img_size,
    check_requirements,
    check_yaml,
    coco80_to_coco91_class,
    colorstr,
    increment_path,
    non_max_suppression,
    print_args,
    scale_boxes,
    xywh2xyxy,
    xyxy2xywh,
)

In [ ]:

class MyComputeLoss:
    sort_obj_iou = False

    # Compute losses
    def __init__(self, cls_pw, obj_pw, na, nc, nl, fl_gamma):
        device = next(model.parameters()).device  # get model device


        # Define criteria
        BCEcls = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([cls_pw], device=device))
        BCEobj = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([obj_pw], device=device))

        # Class label smoothing https://arxiv.org/pdf/1902.04103.pdf eqn 3
        self.cp, self.cn = smooth_BCE(eps=0.0)  # positive, negative BCE targets -> disabled!

        # Focal loss
        g = fl_gamma  # focal loss gamma
        if g > 0:
            BCEcls, BCEobj = FocalLoss(BCEcls, g), FocalLoss(BCEobj, g)


        self.balance = {3: [4.0, 1.0, 0.4]}.get(m.nl, [4.0, 1.0, 0.25, 0.06, 0.02])  # P3-P7
        self.ssi = 0 # same as yolo code: autobalance=False
        self.BCEcls, self.BCEobj, self.gr = BCEcls, BCEobj, 1.0
        self.na = na  # number of anchors
        self.nc = nc  # number of classes
        self.nl = nl  # number of layers
        self.anchors = m.anchors
        self.device = device

    def __call__(self, p, targets):  # predictions, targets
        lcls = torch.zeros(1, device=self.device)  # class loss
        lbox = torch.zeros(1, device=self.device)  # box loss
        lobj = torch.zeros(1, device=self.device)  # object loss
        tcls, tbox, indices, anchors = self.build_targets(p, targets)  # targets

        # Losses
        for i, pi in enumerate(p):  # layer index, layer predictions
            b, a, gj, gi = indices[i]  # image, anchor, gridy, gridx
            tobj = torch.zeros(pi.shape[:4], dtype=pi.dtype, device=self.device)  # target obj

            n = b.shape[0]  # number of targets
            if n:
                # pxy, pwh, _, pcls = pi[b, a, gj, gi].tensor_split((2, 4, 5), dim=1)  # faster, requires torch 1.8.0
                pxy, pwh, _, pcls = pi[b, a, gj, gi].split((2, 2, 1, self.nc), 1)  # target-subset of predictions

                # Regression
                pxy = pxy.sigmoid() * 2 - 0.5
                pwh = (pwh.sigmoid() * 2) ** 2 * anchors[i]
                pbox = torch.cat((pxy, pwh), 1)  # predicted box
                iou = bbox_iou(pbox, tbox[i], CIoU=True).squeeze()  # iou(prediction, target)
                lbox += (1.0 - iou).mean()  # iou loss

                # Objectness
                iou = iou.detach().clamp(0).type(tobj.dtype)
                if self.sort_obj_iou:
                    j = iou.argsort()
                    b, a, gj, gi, iou = b[j], a[j], gj[j], gi[j], iou[j]
                if self.gr < 1:
                    iou = (1.0 - self.gr) + self.gr * iou
                tobj[b, a, gj, gi] = iou  # iou ratio

                # Classification
                if self.nc > 1:  # cls loss (only if multiple classes)
                    t = torch.full_like(pcls, self.cn, device=self.device)  # targets
                    t[range(n), tcls[i]] = self.cp
                    lcls += self.BCEcls(pcls, t)  # BCE

                # Append targets to text file
                # with open('targets.txt', 'a') as file:
                #     [file.write('%11.5g ' * 4 % tuple(x) + '\n') for x in torch.cat((txy[i], twh[i]), 1)]

            obji = self.BCEobj(pi[..., 4], tobj)
            lobj += obji * self.balance[i]  # obj loss
            if self.autobalance:
                self.balance[i] = self.balance[i] * 0.9999 + 0.0001 / obji.detach().item()

        if self.autobalance:
            self.balance = [x / self.balance[self.ssi] for x in self.balance]
        lbox *= self.hyp["box"]
        lobj *= self.hyp["obj"]
        lcls *= self.hyp["cls"]
        bs = tobj.shape[0]  # batch size

        return (lbox + lobj + lcls) * bs, torch.cat((lbox, lobj, lcls)).detach()

    def build_targets(self, p, targets):
        # Build targets for compute_loss(), input targets(image,class,x,y,w,h)
        na, nt = self.na, targets.shape[0]  # number of anchors, targets
        tcls, tbox, indices, anch = [], [], [], []
        gain = torch.ones(7, device=self.device)  # normalized to gridspace gain
        ai = torch.arange(na, device=self.device).float().view(na, 1).repeat(1, nt)  # same as .repeat_interleave(nt)
        targets = torch.cat((targets.repeat(na, 1, 1), ai[..., None]), 2)  # append anchor indices

        g = 0.5  # bias
        off = (
            torch.tensor(
                [
                    [0, 0],
                    [1, 0],
                    [0, 1],
                    [-1, 0],
                    [0, -1],  # j,k,l,m
                    # [1, 1], [1, -1], [-1, 1], [-1, -1],  # jk,jm,lk,lm
                ],
                device=self.device,
            ).float()
            * g
        )  # offsets

        for i in range(self.nl):
            anchors, shape = self.anchors[i], p[i].shape
            gain[2:6] = torch.tensor(shape)[[3, 2, 3, 2]]  # xyxy gain

            # Match targets to anchors
            t = targets * gain  # shape(3,n,7)
            if nt:
                # Matches
                r = t[..., 4:6] / anchors[:, None]  # wh ratio
                j = torch.max(r, 1 / r).max(2)[0] < self.hyp["anchor_t"]  # compare
                # j = wh_iou(anchors, t[:, 4:6]) > model.hyp['iou_t']  # iou(3,n)=wh_iou(anchors(3,2), gwh(n,2))
                t = t[j]  # filter

                # Offsets
                gxy = t[:, 2:4]  # grid xy
                gxi = gain[[2, 3]] - gxy  # inverse
                j, k = ((gxy % 1 < g) & (gxy > 1)).T
                l, m = ((gxi % 1 < g) & (gxi > 1)).T
                j = torch.stack((torch.ones_like(j), j, k, l, m))
                t = t.repeat((5, 1, 1))[j]
                offsets = (torch.zeros_like(gxy)[None] + off[:, None])[j]
            else:
                t = targets[0]
                offsets = 0

            # Define
            bc, gxy, gwh, a = t.chunk(4, 1)  # (image, class), grid xy, grid wh, anchors
            a, (b, c) = a.long().view(-1), bc.long().T  # anchors, image, class
            gij = (gxy - offsets).long()
            gi, gj = gij.T  # grid indices

            # Append
            indices.append((b, a, gj.clamp_(0, shape[2] - 1), gi.clamp_(0, shape[3] - 1)))  # image, anchor, grid
            tbox.append(torch.cat((gxy - gij, gwh), 1))  # box
            anch.append(anchors[a])  # anchors
            tcls.append(c)  # class

        return tcls, tbox, indices, anch


In [3]:
cuda = False
if cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
half = False
weights = "/saion/Deploy/beachbot/beach-cleaning-object-detection/Models/beachbot_yolov5s_beach-cleaning-object-detection__v3-augmented_ver__2__yolov5pytorch_1280/best.pt"
data = "??"
imgsz=640 #??

# Load model
model = DetectMultiBackend(weights, device=device, dnn=False, data=None, fp16=half)

#model.hyp = hyp

stride, pt, jit, engine = model.stride, model.pt, model.jit, model.engine
imgsz = check_img_size(imgsz, s=stride)  # check image size
half = model.fp16  # FP16 supported on limited backends with CUDA

model.eval()

#compute_loss = ComputeLoss(model)


Fusing layers... 
beachbot_YOLOv5s summary: 157 layers, 7026307 parameters, 0 gradients, 15.8 GFLOPs


DetectMultiBackend(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (2): C3(
        (cv1): Conv(
          (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (cv3): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (m): Sequential(
          (0): Bottleneck(
            (cv1): Conv(
              (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
              (act): SiLU(inplace=True)
            )
            (cv2): Conv(

In [ ]:


if cuda:
    im = im.to(device, non_blocking=True)
    targets = targets.to(device)
im = im.half() if half else im.float()  # uint8 to fp16/32
im /= 255  # 0 - 255 to 0.0 - 1.0
nb, _, height, width = im.shape  # batch size, channels, height, width


pred, train_out = model(im)
#xywhn2xyxy(x, w=640, h=640, padw=0, padh=0)
loss, loss_items = compute_loss(pred, targets.to(device))  # loss scaled by batch_size

In [4]:
dataset_location = "/saion/Deploy/beachbot/beach-cleaning-object-detection/Datasets/beach-cleaning-object-detection.v8-yolotrain.yolov5pytorch/"
import yaml
with open(dataset_location + "/data.yaml", 'r') as stream:
    data_cfg = yaml.safe_load(stream)
    num_classes = str(data_cfg['nc'])
    list_classes = data_cfg['names']
    print("Dataset defines", num_classes, "classes ->\n", list_classes)
from os import listdir
from os.path import isfile, join

train_folder=dataset_location + "/train"
test_folder=dataset_location + "/valid"

# read list of all image files in dataset folder
train_imgs = [f for f in listdir(train_folder+"/images") if (isfile(join(train_folder+"/images", f)) and f.endswith(".jpg"))]
test_imgs = [f for f in listdir(test_folder+"/images") if (isfile(join(test_folder+"/images", f)) and f.endswith(".jpg"))]


train_data=[]
for f in train_imgs:
    f_meta = join(train_folder+"/labels", f.replace(".jpg",".txt"))
    meta=[]
    if isfile(f_meta):
        with open(f_meta) as fc:
            lines = [line.rstrip() for line in fc]
            for line in lines:
                content = [float(value) for value in line.split(' ')]
                if len(content)==5:
                # classid, rect_x, rect_y rect_x2, rect_y2
                    content[0] = int(content[0])
                    meta.append(content)
        if len(meta)>0:
            train_data.append([join(train_folder+"/images", f), meta])

            
            
test_data=[]
for f in test_imgs:
    f_meta = join(test_folder+"/labels", f.replace(".jpg",".txt"))
    meta=[]
    if isfile(f_meta):
        with open(f_meta) as fc:
            lines = [line.rstrip() for line in fc]
            for line in lines:
                content = [float(value) for value in line.split(' ')]
                if len(content)==5:
                # classid, rect_x, rect_y rect_x2, rect_y2
                    content[0] = int(content[0])
                    meta.append(content)
        if len(meta)>0:
            test_data.append([join(train_folder+"/images", f), meta])
            
            
            
print("Num training roi boxes is", len(train_data))
print("Num testing roi boxes is", len(test_data))


Dataset defines 6 classes ->
 ['other_avoid', 'other_avoid_boundaries', 'other_avoid_ocean', 'others_traversable', 'trash_easy', 'trash_hard']
Num training roi boxes is 2688
Num testing roi boxes is 255


In [5]:
data_nr = 3
vis_data=train_data


import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

im = Image.open(vis_data[data_nr][0])
im_class_nr = [meta[0] for meta in vis_data[data_nr][1]]
im_class = [list_classes[clsmeta] for clsmeta in im_class_nr]
im_roi = [meta[1:] for meta in vis_data[data_nr][1]]

im_h = im.height
im_w = im.width

print("Dataset entry",data_nr, "dimension is", im_w, "x", im_h)
for clsname,roidata in zip(im_class, im_roi):
    print("ROI", clsname, "at pos", roidata)



Dataset entry 3 dimension is 1280 x 800
ROI trash_hard at pos [0.471875, 0.244375, 0.16875, 0.21375]
ROI other_avoid_boundaries at pos [0.22265625, 0.140975, 0.26302343750000007, 0.10972500000000002]
ROI other_avoid_boundaries at pos [0.46224218749999996, 0.1173625, 0.2960078125000001, 0.04583749999999998]


In [6]:

img_in = np.array(im).T
img_in = img_in.swapaxes(-1,-2)
img_in = img_in[None,:,:,:]
img_in = torch.tensor(img_in)

if cuda:
    img_in = img_in.to(device, non_blocking=True)
    targets = targets.to(device)
img_in = img_in.half() if half else img_in.float()  # uint8 to fp16/32
img_in /= 255  # 0 - 255 to 0.0 - 1.0
nb, _, height, width = img_in.shape  # batch size, channels, height, width


pred, train_out = model(img_in)

In [7]:
train_out


[tensor([[[[[ 4.07594e-01, -2.98842e-01, -1.54125e-01,  ..., -3.72892e+00, -3.31960e+00, -3.09843e+00],
            [ 4.00249e-01, -4.36838e-01,  1.05637e+00,  ..., -3.81731e+00, -3.48133e+00, -3.42772e+00],
            [ 1.95598e-01, -4.88688e-01,  1.87738e+00,  ..., -3.49302e+00, -3.33366e+00, -3.39955e+00],
            ...,
            [ 3.76696e-01, -4.29538e-01,  1.85353e+00,  ..., -3.55396e+00, -3.31198e+00, -3.15186e+00],
            [ 3.40237e-01, -3.06981e-01,  1.12277e+00,  ..., -3.81463e+00, -3.54311e+00, -3.25018e+00],
            [-1.41531e-01, -7.50973e-01,  2.98337e-01,  ..., -4.03919e+00, -3.23591e+00, -2.59866e+00]],
 
           [[ 2.95415e-01, -4.43952e-01, -1.78711e-01,  ..., -4.39768e+00, -2.88237e+00, -2.89815e+00],
            [-1.88489e-01, -8.04545e-01,  1.05595e+00,  ..., -4.00422e+00, -3.49144e+00, -3.49244e+00],
            [-4.04011e-01, -1.08446e+00,  2.38610e+00,  ..., -3.55194e+00, -3.59479e+00, -3.66406e+00],
            ...,
            [ 1.94393e-01, 

In [8]:
import beachbot
dataset = beachbot.ai.Dataset("/saion/Deploy/beachbot/beach-cleaning-object-detection/Datasets/beach-cleaning-object-detection.v8-yolotrain.yolov5pytorch/")

2024-09-09 15:59:33,851 - beachbot - WARNING - Jetson GPIO library not installed or not available! Motor interface may not be available!
2024-09-09 15:59:33,852 - beachbot - WARNING - Jetson utils not installed or not available! JetsonGstCameraNative not available!
2024-09-09 15:59:33,853 - beachbot - INFO - Added class <class 'beachbot.ai.yolov5_onnx.Yolo5Onnx'> with type YOLOv5
2024-09-09 15:59:33,854 - beachbot - INFO - Added class <class 'beachbot.ai.yolov5_opencv.Yolo5OpenCV'> with type YOLOv5
2024-09-09 15:59:33,855 - beachbot - INFO - Added class <class 'beachbot.ai.yolov5_torch_hub.Yolo5TorchHub'> with type YOLOv5_Torch_Hub
2024-09-09 15:59:33,855 - beachbot - INFO - Added class <class 'beachbot.ai.yolov5_torch_hub.Yolo5TorchHub'> with type YOLOv5
2024-09-09 15:59:33,856 - beachbot - INFO - Added class <class 'beachbot.ai.ssd_mobilenet_torchvision.SSDMobileNetTorchvision'> with type SSDMobilenet_Torchvision
2024-09-09 15:59:33,858 - beachbot - WARNING - Jetson utils not install

In [26]:
imnr = 3

imgpath = dataset.images[imnr]
rects = dataset.rects[imnr]

In [27]:
im = Image.open(imgpath)
im_class_nr = [r['classid'] for r in rects]
im_class = [dataset.classes[r['classid']] for r in rects]
im_roi = [r['rect'] for r in rects]

im_h = im.height
im_w = im.width

print("Dataset entry",data_nr, "dimension is", im_w, "x", im_h)
for clsname,roidata in zip(im_class, im_roi):
    print("ROI", clsname, "at pos", roidata)


Dataset entry 3 dimension is 1280 x 800
ROI trash_hard at pos [0.471875, 0.244375, 0.16875, 0.21375]
ROI other_avoid_boundaries at pos [0.22265625, 0.140975, 0.26302343750000007, 0.10972500000000002]
ROI other_avoid_boundaries at pos [0.46224218749999996, 0.1173625, 0.2960078125000001, 0.04583749999999998]


In [28]:
# model_paths = beachbot.ai.DerbrisDetector.list_model_paths()
# print("Model paths are", model_paths)
model_path = "/saion/Deploy/beachbot/beach-cleaning-object-detection/Models/beachbot_yolov5s_beach-cleaning-object-detection__v3-augmented_ver__2__yolov5pytorch_1280/"
print("Model path is", model_path)

model_file = model_path+os.path.sep+"best.onnx"

model_type = beachbot.ai.DerbrisDetector.get_model_type(model_path)
print("Model type is", model_type)

model_cls_list= beachbot.ai.DerbrisDetector.list_models_by_type(model_type)
print("Model classes are", model_cls_list)

model_cls = model_cls_list[0]
ai_detect = model_cls(model_file=model_file, use_accel=False)

Model path is /saion/Deploy/beachbot/beach-cleaning-object-detection/Models/beachbot_yolov5s_beach-cleaning-object-detection__v3-augmented_ver__2__yolov5pytorch_1280/
Model type is YOLOv5
Model classes are [<class 'beachbot.ai.yolov5_onnx.Yolo5Onnx'>, <class 'beachbot.ai.yolov5_opencv.Yolo5OpenCV'>, <class 'beachbot.ai.yolov5_torch_hub.Yolo5TorchHub'>]
2024-09-09 16:14:11,768 - beachbot - INFO - Exported ONNX model operates on images of size 1280x800 [wxh] pixels
2024-09-09 16:14:11,768 - beachbot - INFO - Dataset defines 6 classes -> ['other_avoid', 'other_avoid_boundaries', 'other_avoid_ocean', 'others_traverable', 'trash_easy', 'trash_hard']
2024-09-09 16:14:11,769 - beachbot - INFO - No Gpu acceleration availabe!
2024-09-09 16:14:11,769 - beachbot - INFO - DL providers are:['CPUExecutionProvider']
2024-09-09 16:14:11,830 - beachbot - INFO - model type is tensor(float16)


In [33]:

frame = np.array(im, dtype=ai_detect.dtype)
result = ai_detect.apply_model_percent(frame)


In [36]:
import torch.nn as nn
import torch.nn.functional as F
import torch




def hard_negative_mining(loss, labels, neg_pos_ratio):
    """
    It used to suppress the presence of a large number of negative prediction.
    It works on image level not batch level.
    For any example/image, it keeps all the positive predictions and
     cut the number of negative predictions to make sure the ratio
     between the negative examples and positive examples is no more
     the given ratio for an image.

    Args:
        loss (N, num_priors): the loss for each example.
        labels (N, num_priors): the labels.
        neg_pos_ratio:  the ratio between the negative examples and positive examples.
    """
    pos_mask = labels > 0
    num_pos = pos_mask.long().sum(dim=1, keepdim=True)
    num_neg = num_pos * neg_pos_ratio

    loss[pos_mask] = -math.inf
    _, indexes = loss.sort(dim=1, descending=True)
    _, orders = indexes.sort(dim=1)
    neg_mask = orders < num_neg
    return pos_mask | neg_mask

class MultiboxLoss(nn.Module):
    def __init__(self, priors, iou_threshold, neg_pos_ratio,
                 center_variance, size_variance, device):
        """Implement SSD Multibox Loss.

        Basically, Multibox loss combines classification loss
         and Smooth L1 regression loss.
        """
        super(MultiboxLoss, self).__init__()
        self.iou_threshold = iou_threshold
        self.neg_pos_ratio = neg_pos_ratio
        self.center_variance = center_variance
        self.size_variance = size_variance
        self.priors = priors
        self.priors.to(device)

    def forward(self, confidence, predicted_locations, labels, gt_locations):
        """Compute classification loss and smooth l1 loss.

        Args:
            confidence (batch_size, num_priors, num_classes): class predictions.
            locations (batch_size, num_priors, 4): predicted locations.
            labels (batch_size, num_priors): real labels of all the priors.
            boxes (batch_size, num_priors, 4): real boxes corresponding all the priors.
        """
        num_classes = confidence.size(2)
        with torch.no_grad():
            # derived from cross_entropy=sum(log(p))
            loss = -F.log_softmax(confidence, dim=2)[:, :, 0]
            mask = hard_negative_mining(loss, labels, self.neg_pos_ratio)

        confidence = confidence[mask, :]
        classification_loss = F.cross_entropy(confidence.reshape(-1, num_classes), labels[mask], size_average=False)
        pos_mask = labels > 0
        predicted_locations = predicted_locations[pos_mask, :].reshape(-1, 4)
        gt_locations = gt_locations[pos_mask, :].reshape(-1, 4)
        smooth_l1_loss = F.smooth_l1_loss(predicted_locations, gt_locations, size_average=False)
        num_pos = gt_locations.size(0)
        return smooth_l1_loss/num_pos, classification_loss/num_pos

In [55]:
def np_iou(boxes1, boxes2):
    x11, y11, x12, y12 = np.split(boxes1, 4, axis=1)
    x21, y21, x22, y22 = np.split(boxes2, 4, axis=1)
    xA = np.maximum(x11, np.transpose(x21))
    yA = np.maximum(y11, np.transpose(y21))
    xB = np.minimum(x12, np.transpose(x22))
    yB = np.minimum(y12, np.transpose(y22))
    interArea = np.maximum((xB - xA + 1), 0) * np.maximum((yB - yA + 1), 0)
    boxAArea = (x12 - x11 + 1) * (y12 - y11 + 1)
    boxBArea = (x22 - x21 + 1) * (y22 - y21 + 1)
    iou = interArea / (boxAArea + np.transpose(boxBArea) - interArea)
    return iou 

def boxes_xywh_to_xyxy(boxes):
    res = np.copy(boxes[:,:])
    res[:,2] += res[:,0]
    res[:,3] += res[:,1]
    return res

In [56]:
box_pred = boxes_xywh_to_xyxy(np.stack(result[2]))
box_gt = boxes_xywh_to_xyxy(np.stack(im_roi))


In [69]:
ior_res = np_iou(box_pred, box_gt)

In [117]:
ior_res

array([[    0.74394,     0.74893,     0.67153],
       [    0.60536,      0.5595,      0.8128],
       [    0.55207,      0.4824,     0.70314],
       [    0.42513,     0.31689,     0.50126],
       [    0.39823,     0.70187,     0.44806]])

In [101]:
ior_res.max(axis=1)
np_iou(box_gt, box_gt)

array([[          1,      0.5668,     0.63165],
       [     0.5668,           1,      0.6115],
       [    0.63165,      0.6115,           1]])

In [124]:
def calc_loss(box_pred, label_pred, conf_pred, box_gt, label_gt, iou_thresh=0.5):
    iou_res = np_iou(box_pred, box_gt)
    num_detected = box_pred.shape[0]
    num_gt = box_gt.shape[0]

    unfoud_classes = list(range(num_gt))

    

    best_fitting_gt = iou_res.argmax(axis=1)
    FP=0
    FN=0
    TP=0
    reg_loss=0.0
    class_err=0.0
    for bnum in range(num_detected):
        gt_num = best_fitting_gt[bnum]
        reg_loss += 1.0 - iou_res[bnum, gt_num]
        if iou_res[bnum, gt_num]>iou_thresh:
            TP+=1
            if label_pred[bnum]!=label_gt[gt_num]:
                class_err+=1
            if gt_num in unfoud_classes:
                unfoud_classes.remove(gt_num)
        else:
            FP +=1
    reg_loss /= num_detected
    class_err /= TP
    FN = len(unfoud_classes)

    return class_err, reg_loss, (TP, FP, FN)




In [125]:

box_pred = boxes_xywh_to_xyxy(np.stack(result[2]))
box_gt = boxes_xywh_to_xyxy(np.stack(im_roi))
label_pred = result[0]
conf_pred = result[1]
label_gt = im_class_nr


calc_loss(box_pred, label_pred, conf_pred, box_gt, label_gt)

(0.8, 0.30640199807571084, (5, 0, 1))

In [110]:
result[0]

[5, 2, 2, 2, 1]

In [118]:
im_class_nr.

[5, 1, 1]

In [119]:
2/5

0.4

In [80]:
ior_res.argmax(axis=0)

array([0, 0, 1])

In [82]:
ior_res

array([[    0.74394,     0.74893,     0.67153],
       [    0.60536,      0.5595,      0.8128],
       [    0.55207,      0.4824,     0.70314],
       [    0.42513,     0.31689,     0.50126],
       [    0.39823,     0.70187,     0.44806]])

In [83]:
box_pred

array([[    0.38306,      0.1343,     0.55991,     0.37133],
       [    0.55771,    0.044609,     0.76494,     0.24148],
       [    0.64561,    0.049844,     0.84033,     0.27453],
       [    0.87734,     0.10133,      1.0039,     0.34742],
       [   0.025879,    0.096602,      0.2915,     0.24027]])

In [85]:
box_gt

array([[    0.47187,     0.24438,     0.64062,     0.45813],
       [    0.22266,     0.14097,     0.48568,      0.2507],
       [    0.46224,     0.11736,     0.75825,      0.1632]])

In [103]:
result[1]

[0.779, 0.4365, 0.401, 0.3694, 0.2211]

In [112]:
im_class_nr

[5, 1, 1]

In [111]:
im_class

['trash_hard', 'other_avoid_boundaries', 'other_avoid_boundaries']

In [90]:
calc:

- true pos
(- true neg)
- false pos 
- false neg

SyntaxError: invalid syntax (702639152.py, line 1)

In [94]:
# how to calc?
# fore ach found class, search best matching class < htresh iou
# if none - > false negative
# if yes -> check class assignment -> wrong? fa

In [106]:
im_class_nr

[5, 1, 1]